# Preparation

In [ ]:
%pip install -U googlemaps


In [ ]:
import os
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import contextily as ctx
from datetime import datetime
import time
import numpy as np
import googlemaps
import seaborn

# Import MTA Turnstile Data

In [ ]:
mta = pd.DataFrame(data=None)
mta_root = "Data/MTA/2021_Weekly_Raw/"
for item in os.listdir(mta_root):
    if not item.startswith('.') and os.path.isfile(os.path.join(mta_root, item)):
        mta = pd.concat([mta, pd.read_csv('Data/MTA/2021_Weekly_Raw/'+item)])

In [ ]:
mta

In [ ]:
mta.columns = ['C/A', 'Unit', 'Scp', 'Station', 'Linename', 'Division', 'Date', 'Time', 'Desc', 'Entries', 'Exits']
mta

# Data cleaning: Processing Exit/Entrance Numbers and Time

In [ ]:
def date_convert(date_to_convert):
     return time.mktime(datetime.strptime(date_to_convert, "%m/%d/%Y %H:%M:%S").timetuple())

In [ ]:
dt = mta['Date'] + ' ' + mta['Time']
dt

In [ ]:
mta['Timestamp'] = dt.apply(date_convert)
mta

In [ ]:
mta = mta.sort_values(by=['Station','C/A','Scp','Timestamp'],ascending=(True, True,True, True))

In [ ]:
mta.reset_index(drop=True, inplace=True)
mta

In [ ]:
start_index = np.array([0])
for i in mta.index[1:len(mta.index)]:
    if((mta.loc[i,'Station']==mta.loc[i-1,'Station']) & (mta.loc[i,'C/A']==mta.loc[i-1,'C/A']) & (mta.loc[i,'Scp']==mta.loc[i-1,'Scp'])):
        mta.loc[i,'Entries_n'] = mta.loc[i,'Entries'] - mta.loc[i-1,'Entries']
        mta.loc[i,'Exits_n'] = mta.loc[i,'Exits'] - mta.loc[i-1,'Exits']
    else:
        start_index = np.append(start_index, i)

In [ ]:
for i in start_index:
    mta.loc[i,'Entries_n'] = 0
    mta.loc[i,'Exits_n'] = 0
mta

In [ ]:
mta.loc[311990:312000]

In [ ]:
### Data Cleaning
### Abs
mta['Entries_n'] = abs(mta['Entries_n'])
mta['Exits_n'] = abs(mta['Exits_n'])

In [ ]:
### Remove extreme values
mta = mta[(mta['Entries_n']<=10000) & (mta['Exits_n']<=10000)]

In [ ]:
### Export
mta.to_csv('Data/MTA/MTA_ALL_RAW.csv')

In [ ]:
mta = pd.read_csv(('Data/MTA/MTA_ALL_RAW.csv'))
mta.head()

In [ ]:
### Divide 4-hour intervals into 4 one-hour intervels
time_interval = 3600
mta.loc[:,'Exits_n'] = mta.loc[:,'Exits_n']/4
mta.loc[:,'Entries_n'] = mta.loc[:,'Entries_n']/4

mta_1 = mta.copy(deep=True)
mta_2 = mta.copy(deep=True)
mta_3 = mta.copy(deep=True)
mta_1['Timestamp'] = mta_1['Timestamp'] - time_interval * 1
mta_2['Timestamp'] = mta_2['Timestamp'] - time_interval * 2
mta_3['Timestamp'] = mta_3['Timestamp'] - time_interval * 3

In [ ]:
mta

In [ ]:
### Drop Weekends
mta = mta[(mta['Timestamp'].apply(lambda x: time.localtime(x).tm_wday)!=5) & (mta['Timestamp'].apply(lambda x: time.localtime(x).tm_wday)!=6)]
mta = mta[(mta['Timestamp'].apply(lambda x: time.localtime(x).tm_wday)!=5) & (mta['Timestamp'].apply(lambda x: time.localtime(x).tm_wday)!=6)]
mta = mta[(mta['Timestamp'].apply(lambda x: time.localtime(x).tm_wday)!=5) & (mta['Timestamp'].apply(lambda x: time.localtime(x).tm_wday)!=6)]
mta = mta[(mta['Timestamp'].apply(lambda x: time.localtime(x).tm_wday)!=5) & (mta['Timestamp'].apply(lambda x: time.localtime(x).tm_wday)!=6)]

In [ ]:
mta

In [ ]:
def get_hour(timestamp):
    return time.localtime(timestamp).tm_hour + round(time.localtime(timestamp).tm_min/60)


In [ ]:
mta['Time_in_a_day'] = mta['Timestamp'].apply(get_hour)

In [ ]:
mta[mta['Time_in_a_day']==24] = 0
mta['Time_in_a_day'].unique()

In [ ]:
mta = mta.sort_values(by=['Station','C/A','Scp','Timestamp'],ascending=(True, True,True, True))
mta.reset_index(drop=True, inplace=True)
mta

# Compress the Data to Station Level

In [ ]:
mta_compressed = pd.DataFrame(data=None)
for i in range(0,len(mta.index)):
    print (i)

In [ ]:
mta_compressed = pd.DataFrame(data=None)
for unit in mta['Unit']:
    data = mta[mta['Unit']==unit]
    mta_compressed['Unit'] = unit
    for i in range (0,24):
        mta_compressed['Entries_'+str(i)] = data[data['Time_in_a_day']==i]['Entries_n'].sum()
        mta_compressed['Exits_'+str(i)] = data[data['Time_in_a_day']==i]['Exits_n'].sum()



In [ ]:
mta_compressed

# Link Geo Location to the Turnstile Data

In [ ]:
### Link Stations
### Thanks to Chris Whong: https://medium.com/qri-io/taming-the-mtas-unruly-turnstile-data-c945f5f96ba0
Booth_code = pd.read_csv('Data/MTA/Stations/remote_complex_lookup.csv')
Booth_code.columns = ['remote', 'booth', 'complex_id', 'station', 'line_name', 'division']
Booth_code

In [ ]:
mta = mta.merge(Booth_code[['remote','complex_id']], left_on='Unit', right_on='remote', how='left')

In [ ]:
mta

In [ ]:
Station_data = pd.read_csv('Data/MTA/Stations/Stations.csv')
Station_data

In [ ]:
mta = mta.merge(Station_data[['Complex ID','GTFS Latitude','GTFS Longitude']], left_on='complex_id', right_on='Complex ID', how='left')